<a href="https://colab.research.google.com/github/R12942159/NTU_DLCV/blob/Hw2/p3_ModelA_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch


class GRL(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, constant):

In [ ]:
from torch import nn

class Extractor(nn.Module):
    def __init__(self) -> None:
        super(Extractor, self).__init__()
        self.extractor = nn.Sequential